Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
!unzip data.zip
!pip install transformers
!pip install datasets

--2021-10-21 20:16:08--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-21 20:16:09--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   115KB/s    in 1.8s    

2021-10-21 20:16:12 (115 KB/s) - ‘data.zip’ saved [217313/217313]



In [1]:
import os
import time
import random
import datetime
import numpy as np
import pandas
from pdb import set_trace
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_metric

In [7]:
## Various utility functions
class args_class:
    dataset = "data/"
    model_name = "bert-base-uncased"
    gpu = 0
    batch_size = 8
    results_dir = "2018EE10957_B_model"
    epochs = 3

def set_seed(seed=4):
	# fixing seeds to reproduce results (exact!)
	# torch.backends.cudnn.deterministic = True
	torch.manual_seed(seed)
	random.seed(seed)
	np.random.seed(seed)

def parse_args():
	return args_class()

def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')

In [8]:
def process_data(path, train=True, test=False):
    if test == True:
        x_test = pandas.read_csv("test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
        word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
        word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
        x_test = x_test.drop("position", axis=1)
        x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
        x_test.insert(3, "word1", word1)
        x_test.insert(5, "word2", word2)
        # x_test.to_csv("./test.csv", header=None, index=False)
        return x_test
    
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    # dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)
    return dataset

class myDataset(Dataset):
    def __init__(self, data_dir, max_len=128, train=True, test=False, model_name='bert-base-uncased'):
        self.data_dir = data_dir
        self.max_len = max_len
        self.train = train
        self.test = test

        # convert raw data into pandas data frame
        self.dataset_df = process_data(data_dir, train=train, test=test)

        # tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name) # , do_lower_case=True)

    def __getitem__(self, index):
        sample = self.dataset_df.iloc[index]

        x = self.tokenizer(sample['sen1'], sample['sen2'], return_tensors='pt', add_special_tokens=True, \
            max_length=self.max_len, padding='max_length', truncation=True)
        if self.test == False:
            y = torch.tensor(sample['label'])

        # format as required by Train
        item = {k: v[0] for k,v in x.items()} # only one datapoint; shape: 128
        if self.test == False:
            item['labels'] = y
        return item
    
    def __len__(self):
        return len(self.dataset_df)

In [9]:
set_seed()

metric = load_metric("accuracy")
def compute_metrics_fn(eval_prediction):
    logits, labels = eval_prediction
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

args = parse_args()
trainset = myDataset(args.dataset, max_len=100, train=True, model_name=args.model_name)
devset = myDataset(args.dataset, max_len=100, train=False, model_name=args.model_name)
model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=2)
print("resource preparation done: {}".format(datetime.datetime.now()))

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/shubham/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home

resource preparation done: 2021-10-25 03:13:34.631836


In [5]:
training_args = TrainingArguments(
    output_dir=args.results_dir, 
    overwrite_output_dir=True, 
    evaluation_strategy="epoch", 
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=args.epochs,
    seed=4,
    per_device_eval_batch_size=args.batch_size,
    per_device_train_batch_size=args.batch_size,
    warmup_steps=500,
    logging_dir='./logs',
    logging_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model=model, args=training_args,
    train_dataset=trainset, eval_dataset=devset,
    compute_metrics=compute_metrics_fn
)

trainer.train()

***** Running training *****
  Num examples = 5428
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2037


Epoch,Training Loss,Validation Loss,Accuracy
1,0.605600,0.707967,0.639498
2,0.400600,0.868928,0.681818
3,0.174100,1.495763,0.688088


***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to 2018EE10957_B_model/checkpoint-679
Configuration saved in 2018EE10957_B_model/checkpoint-679/config.json
Model weights saved in 2018EE10957_B_model/checkpoint-679/pytorch_model.bin
Deleting older checkpoint [2018EE10957_B_model/checkpoint-2037] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to 2018EE10957_B_model/checkpoint-1358
Configuration saved in 2018EE10957_B_model/checkpoint-1358/config.json
Model weights saved in 2018EE10957_B_model/checkpoint-1358/pytorch_model.bin
Deleting older checkpoint [2018EE10957_B_model/checkpoint-679] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to 2018EE10957_B_model/checkpoint-2037
Configuration saved in 2018EE10957_B_model/checkpoint-2037/config.json
Model weights saved in 2018EE10957_B_model/checkpoint-2

TrainOutput(global_step=2037, training_loss=0.39342883432619585, metrics={'train_runtime': 363.2283, 'train_samples_per_second': 44.831, 'train_steps_per_second': 5.608, 'total_flos': 836816489352000.0, 'train_loss': 0.39342883432619585, 'epoch': 3.0})

### testing

In [11]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2021-10-25 03:14:06--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to 10.10.78.22:3128... connected.
Proxy request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-25 03:14:06--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to 10.10.78.22:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  --.-KB/s    in 0.005s  

2021-10-25 03:14:06 (12.2 MB

In [12]:
testset = myDataset(args.dataset, max_len=100, train=False, test=True, model_name=args.model_name)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/shubham/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home

In [32]:
model = AutoModelForSequenceClassification.from_pretrained("2018EE10957_B_model/checkpoint-2037/", num_labels=2)
trainer = Trainer(
    model=model
)

loading configuration file 2018EE10957_B_model/checkpoint-2037/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file 2018EE10957_B_model/checkpoint-2037/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the w

In [35]:
out = trainer.predict(testset)
predictions = np.argmax(out[0], axis=-1)
predictions = pandas.DataFrame(predictions)[0].apply(lambda label : "T" if label == 1 else "F")
predictions.to_csv("output.txt", header=None, index=None)
print("predictions written in output.txt")

predictions written in output.txt


In [36]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
    prediction, gold = prediction.strip(), gold.strip()
    total += 1
    if prediction == gold:
        correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2021-10-25 03:27:23--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to 10.10.78.22:3128... connected.
Proxy request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-25 03:27:23--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to 10.10.78.22:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt.1’

test.gold.txt.1     100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-25 03:27:23 (23.6 

## debugging

In [ ]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2018EE10957_A_model.zip 2018EE10957_B_model

## Upload it to Google drive and ensure that the testing notebook uses the correct link

In [9]:
# dataset_df = process_data("data", train=False)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
# dataset_df.iloc[12:14]['sen1'].values
# sample = dataset_df.iloc[12:14]
# x = tokenizer(list(sample['sen1']), list(sample['sen2']), return_tensors='pt', \
#                         add_special_tokens=True, max_length=128, padding='max_length')
# y = torch.tensor(list(sample['label']))
# for k,v in x.items():
#     print(v.shape)

In [11]:
# import numpy as np
# from datasets import load_metric

# metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [20]:
# metric

In [12]:
# import transformers
# transformers.logging.set_verbosity(40)

In [16]:
# m

TrainOutput(global_step=114, training_loss=0.6710249917548999, metrics={'train_runtime': 48.3189, 'train_samples_per_second': 112.337, 'train_steps_per_second': 2.359, 'total_flos': 278938829784000.0, 'train_loss': 0.6710249917548999, 'epoch': 1.0})

In [58]:
# training_args = TrainingArguments(
#     output_dir=task.args.results_dir, 
#     overwrite_output_dir=True, 
#     evaluation_strategy="epoch", 
#     learning_rate=5e-5,
#     num_train_epochs=3, 
#     seed=4,
#     per_device_eval_batch_size=task.args.batch_size,
#     per_device_train_batch_size=task.args.batch_size,
# )

# trainer = Trainer(
#     model=task.model, args=training_args,
#     train_dataset=task.trainset, eval_dataset=task.devset,
# )

# trainer.train()

In [1]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id, tokenizer.mask_token_id)
# tokenizer.encode(["hey my name is shubham", "they 'll play this game very hastily"], add_special_tokens=True)
# print(tokenizer.convert_tokens_to_ids(["[PAD]", "[SEP]", "[CLS]", "[MASK]"]))
# print(tokenizer.convert_ids_to_tokens([0,100,101, 102, 103, 104]))

In [3]:
# df = process_data("./data/", train=False)
# df.iloc[0]

In [19]:
# out = tokenizer(["heyhey myhey","hi","hiello"], add_special_tokens=False, return_tensors='pt', padding='max_length', max_length=5, truncation=True)
# out

In [10]:
# tokens = tokenizer.convert_ids_to_tokens(out['input_ids'][0])
# print(tokens)
# sen = tokenizer.convert_tokens_to_string(tokens)
# print(sen)
# print(" ".join(tokens))

In [20]:
# from transformers import AutoModelForSequenceClassification
# bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# bert = bert.to('cuda')
# trainset = WiC_dataset("data", max_len=128, train=True, tokenizer='bert-base-uncased')
# trainloader = DataLoader(trainset, batch_size = 32, shuffle = True)
# out = bert(x)

In [26]:
# df = process_data("./data/", train=True)
# max_num_tok = 0
# for i in range(len(df)):
#     t = tokenizer(df.loc[i,'sen1'], df.loc[i,'sen2'], add_special_tokens=True, return_tensors='pt')['input_ids']
#     max_num_tok = max(max_num_tok, t.shape[-1])
# max_num_tok

69

In [29]:
# tokens = tokenizer.convert_ids_to_tokens(t[0])
# print(tokens)
# sen = tokenizer.convert_tokens_to_string(tokens)
# print(sen)
# print(" ".join(tokens))
# tokenizer.decode(t[0])

'[CLS] an invasion of bees. [SEP] an invasion of mobile phones. [SEP]'

In [5]:
# def process_data(path, train=True):
#     train = "train" if train else "validation"
#     x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
#     y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
#     dataset = pandas.concat((x_train, y_train), axis=1)
#     word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
#     word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
#     dataset = dataset.drop("position", axis=1)
#     dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
#     dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
#     dataset.insert(3, "word1", word1)
#     dataset.insert(5, "word2", word2)
#     # dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)
#     return dataset

# class myDataset(Dataset):
#     def __init__(self, data_dir, max_len=128, train=True, model_name='bert-base-uncased'):
#         self.data_dir = data_dir
#         self.max_len = max_len
#         self.train = train

#         # convert raw data into pandas data frame
#         self.dataset_df = process_data(data_dir, train=train)

#         # tokenizer
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

#     def __getitem__(self, index):
#         sample = self.dataset_df.iloc[index]

#         x = self.tokenizer(sample['sen1'], sample['sen2'], return_tensors='pt', add_special_tokens=True, \
#             max_length=self.max_len, padding='max_length', truncation=True)
#         y = torch.tensor(sample['label'])

#         # format as required by Train
#         item = {k: v[0] for k,v in x.items()} # only one datapoint; shape: 128
#         item['labels'] = y
#         return item
    
#     def __len__(self):
#         return len(self.dataset_df)

In [14]:
# from transformers import AutoModel
# bert = AutoModel.from_pretrained("bert-base-cased", num_labels=2)
# bert = bert.to(torch.device('cuda'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# trainset = myDataset("data", max_len=100, train=True, model_name="bert-base-cased")
# devset = myDataset("data", max_len=100, train=False, model_name="bert-base-cased")

# trainloader = DataLoader(trainset, batch_size = 8, shuffle = True)
# devloader = DataLoader(devset, batch_size = 8, shuffle = False)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
# batch=next(iter(trainloader))

In [15]:
# inp={k:v.to(torch.device('cuda')) for k,v in batch.items() if k != 'labels'}
# oup = bert(**inp)

In [23]:
# print(len(oup), oup[0].shape, oup[1].shape)

2 torch.Size([8, 100, 768]) torch.Size([8, 768])


In [26]:
# for p in bert.parameters():
#     p.requires_grad=False

60.18808777429467
predictions written in output.txt
